<div style="text-align: center;">
  <img src="https://github.com/Hack-io-Data/Imagenes/blob/main/01-LogosHackio/logo_naranja@4x.png?raw=true" alt="esquema" />
</div>

# Laboratorio Agrupamiento de Datos

En este laboratorio usaremos el DataFrame creado en la lección anterior resultado de la unión de los conjuntos de datos compartidos contigo. 

**Instrucciones:**

1. Lee cuidadosamente el enunciado de cada ejercicio.

2. Implementa la solución en la celda de código proporcionada.

3. Devuelve los resultados siempre en formato de DataFrame

In [41]:
# Antes de empezar importamos las librerías necesarias
# Para tratamiento de datos
import numpy as np
import pandas as pd
# Para generar todas las combinaciones posibles
import itertools
# Para guardar DataFrames en Excel
from pandas import ExcelWriter

# Ignorar warnings
import warnings
warnings.filterwarnings("ignore")

# Configuración para poder visualizar todas las columnas de los DataFrames
pd.set_option('display.max_columns', None) 

In [42]:
## Cargamos el archivo
df_catalogo = pd.read_pickle("datos/catalogo_completo.pkl")
df_catalogo.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Documentary,2020-10-02,90.0,7.5,English,yes


#### Ejercicio 1: ¿Cuántos títulos hay por cada tipo de contenido ('Movie' o 'TV Show')?

In [43]:
tipo_contenido = df_catalogo.groupby("type").count()
tipo_contenido

,show_id,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Genre,Premiere,Runtime,IMDB Score,Language,Original
type,,,,,,,,,,,,,,,,,
Movie,6131,6131,5943,5656,5691,6131,6131,6129,3503,6131,6131,513,513,513,513,513,513
TV Show,2676,2676,230,2326,2285,2666,2676,2674,491,2676,2676,0,0,0,0,0,0


In [44]:
print(f"Hay {tipo_contenido.loc["Movie", "title"]} de tipo 'Movie' y {tipo_contenido.loc["TV Show", "title"]} de tipo 'TV Show")

Hay 6131 de tipo 'Movie' y 2676 de tipo 'TV Show


#### Ejercicio 2: ¿Cuál es el promedio de puntuación de las películas agrupadas por clasificación de audiencia ('rating')? Interpreta los resultados, ¿hay algo que te llame la atención? Explica por qué. 

In [45]:
puntuacion_rating = round(df_catalogo.groupby("rating")["IMDB Score"].mean(),2)
puntuacion_rating

rating
66 min       NaN
74 min       NaN
84 min       NaN
G           6.50
NC-17        NaN
NR           NaN
PG          6.80
PG-13       6.39
R           6.50
TV-14       6.18
TV-G        6.16
TV-MA       6.10
TV-PG       6.31
TV-Y        6.78
TV-Y7       6.24
TV-Y7-FV     NaN
UR           NaN
Name: IMDB Score, dtype: float64

En primer lugar aparecen 3 valores que no parecen clasificación de audiencia, sino datos de duración.
Hay clasificaciones que no tienen un valor medio, puede deberse a que esas clasificaciones no tengan puntuación en IMDB.
Se puede observar que todos los valores medios obtenidos son bastante similares, entre 6.10 y 6.80, lo que sugiere que no hay categorías especialmente buenas ni especialmente malas.

In [46]:
puntuacion_y_rating = df_catalogo.filter(items = ["rating", "IMDB Score"])
nota_rating = puntuacion_y_rating[puntuacion_y_rating["rating"].isin(["NC-17", "NR", "TV-Y7-FV", "UR"])]
nota_rating .info()

<class 'pandas.core.frame.DataFrame'>
Index: 92 entries, 5264 to 8792
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   rating      92 non-null     object 
 1   IMDB Score  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 2.2+ KB


Hemos creado otro dataframe que contiene solo las clasificaciones que arrojan una media nula y su puntuación en IMDb; al sacar la info vemos que de las 92 filas, las que corresponden a "IMDB Score" son todas nulas.

#### Ejercicio 3: ¿Cuántos títulos hay por cada país? Ordena los resultados de mayor a menor

In [47]:
titulos_pais = (df_catalogo.groupby("country").count()).filter(items =["country", "title"])
# Hemos filtrado para que solo muestre las columnas de país y el conteo de títulos
titulos_pais.sort_values("title", ascending = False)
# Usamos sort_values para ordenar por la columna title, que es la que contiene el conteo y ponemos "ascending = False" para que ordene de forma descencente.

,title
country,
United States,2818
India,972
United Kingdom,419
Japan,245
South Korea,199
...,...
"Australia, Iraq",1
Venezuela,1
"Venezuela, Colombia",1


#### Ejercicio 4: ¿Cuál es la calificación promedio en IMDb de los títulos agrupados por tipo ('Movie' o 'TV Show')?

In [48]:
puntuacion_tipo = df_catalogo.groupby("type")["IMDB Score"].mean()
puntuacion_tipo

type
Movie      6.210916
TV Show         NaN
Name: IMDB Score, dtype: float64

Parece que los títulos de tipo 'TV Show' no tienen calificación en IMDb.

In [49]:
puntuacion_y_tipo = df_catalogo.filter(items = ["type", "IMDB Score"])
nota_series = puntuacion_y_tipo[puntuacion_y_tipo["type"]== "TV Show"]
nota_series.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2676 entries, 1 to 8803
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   type        2676 non-null   object 
 1   IMDB Score  0 non-null      float64
dtypes: float64(1), object(1)
memory usage: 62.7+ KB


Si creamos un DataFrame que únicamente contenga los títulos de tipo "TV Show" y su puntuación en IMDB, podemos ver que tiene 2676 filas y que, para la columna "IMDB Score", todos los valores son nulos.

#### Ejercicio 5: ¿Cuántos títulos se añadieron a Netflix cada año? Para resolver este ejercicio tendrás uqe convertir la columna `date_added` a formato fecha usando el método `pd.to_datetime`. [Aquí](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) tienes la documentación, leeala con detenimiento antes de usar el método. 

In [50]:
# convertimos la columna a formato fecha
df_catalogo["date_added"] = pd.to_datetime(df_catalogo["date_added"], format="mixed")
# creamos columna nueva con el año
df_catalogo["year_added"] = df_catalogo["date_added"].dt.year

In [51]:
# agrupamos los títulos por año
titulos_anual = (df_catalogo.groupby("year_added").count())
# mostramos solo las columnas del año y el conteo de títulos
titulos_anual.filter(items =["year_added", "title"])

,title
year_added,
2008.0,2
2009.0,2
2010.0,1
2011.0,13
2012.0,3
2013.0,11
2014.0,24
2015.0,82
2016.0,429


#### Ejercicio 6: ¿Cuál es el país con más títulos lanzados en el año 2020? Para resolver este ejercicio después del gropuby os puede resultar interesante el método [`idxmax`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.idxmax.html)

In [62]:
# Filtramos los títulos lanzados en 2020
titulos_2020 = df_catalogo[df_catalogo["year_added"] == 2020]
# agrupamos los títulos de 2020 por país contando los títulos
titulos_pais = titulos_2020.groupby("country")["title"].count()

In [61]:
pais_mas_titulos = titulos_pais.idxmax()
pais_mas_titulos

'United States'

#### Ejercicio 7:¿Cuál es el promedio de puntuación en IMDb de los títulos agrupados por país?

In [54]:
df_catalogo["country"] = df_catalogo["country"].str.replace(', France, Algeria', 'France, Algeria')
df_catalogo["country"] = df_catalogo["country"].str.replace(', South Korea', 'South Korea')
puntuacion_pais = df_catalogo.groupby("country")["IMDB Score"].mean()
puntuacion_pais

country
Argentina                                              6.42
Argentina, Brazil, France, Poland, Germany, Denmark     NaN
Argentina, Chile                                        NaN
Argentina, Chile, Peru                                  NaN
Argentina, France                                       NaN
                                                       ... 
Venezuela                                               NaN
Venezuela, Colombia                                     NaN
Vietnam                                                 NaN
West Germany                                            NaN
Zimbabwe                                                NaN
Name: IMDB Score, Length: 746, dtype: float64

#### Ejercicio 8: ¿Cuántos títulos fueron lanzados en cada año agrupados por tipo ('Movie' o 'TV Show')?

In [78]:
titulos_tipo = df_catalogo.groupby(["year_added","type"])["title"].count()
titulos_tipo

year_added  type   
2008.0      Movie         1
            TV Show       1
2009.0      Movie         2
2010.0      Movie         1
2011.0      Movie        13
2012.0      Movie         3
2013.0      Movie         6
            TV Show       5
2014.0      Movie        19
            TV Show       5
2015.0      Movie        56
            TV Show      26
2016.0      Movie       253
            TV Show     176
2017.0      Movie       839
            TV Show     349
2018.0      Movie      1237
            TV Show     412
2019.0      Movie      1424
            TV Show     592
2020.0      Movie      1284
            TV Show     595
2021.0      Movie       993
            TV Show     505
Name: title, dtype: int64

#### Ejercicio 9: ¿Cuál es el país con la mayor cantidad de títulos de clasificación 'TV-MA'?

In [ ]:
# filtramos por los títulos de clasificacion 'TV-MA'
titulos_TVMA = df_catalogo[df_catalogo["rating"] == "TV-MA"]
# agrupamos por país contando los títulos
titulos_TVMA_pais = titulos_TVMA.groupby("country")["title"].count()
# usamos idxmax para sacar el máximo
titulos_TVMA_pais.idxmax()


'United States'

#### Ejercicio 10. ¿Cuál es el promedio de duración de las películas agrupadas por país?

In [85]:
duracion_pais = df_catalogo.groupby("country")["Runtime"].mean()
duracion_pais

country
Argentina                                              96.8
Argentina, Brazil, France, Poland, Germany, Denmark     NaN
Argentina, Chile                                        NaN
Argentina, Chile, Peru                                  NaN
Argentina, France                                       NaN
                                                       ... 
Venezuela                                               NaN
Venezuela, Colombia                                     NaN
Vietnam                                                 NaN
West Germany                                            NaN
Zimbabwe                                                NaN
Name: Runtime, Length: 746, dtype: float64

#### Ejercicio 11: ¿Cuál es el número total de títulos por país y clasificación de audiencia ('rating')?

In [87]:
titulos_pais_rating = df_catalogo.groupby(["country","rating"])["title"].count()
titulos_pais_rating

country       rating
Argentina     NR         2
              R          1
              TV-14      7
              TV-G       2
              TV-MA     39
                        ..
Vietnam       TV-14      3
              TV-G       1
              TV-MA      3
West Germany  TV-MA      1
Zimbabwe      TV-G       1
Name: title, Length: 1174, dtype: int64

#### Ejercicio 12: ¿Cuántos títulos fueron lanzados antes del año 2000 agrupados por país?

In [90]:
# filtramos por los títulos lanzados antes del 2000'
titulos_antes2000 = df_catalogo[df_catalogo["release_year"] < 2000]

# agrupamos por país contando los títulos
titulos_antes2000_pais = titulos_antes2000.groupby("country")["title"].count()

titulos_antes2000_pais

country
Argentina                        3
Argentina, Spain                 1
Australia                        4
Australia, United Kingdom        1
Australia, United States         2
                                ..
United States, Japan             1
United States, Mexico            3
United States, New Zealand       1
United States, United Kingdom    6
West Germany                     1
Name: title, Length: 65, dtype: int64

#### Ejercicio 13: ¿Cuál es la puntuación promedio en IMDb de los títulos lanzados en 2021 agrupados por país?

In [91]:
# filtramos por los títulos lanzados en 2021'
titulos_2021 = df_catalogo[df_catalogo["release_year"] == 2021]
# agrupamos por país contando los títulos
titulos_2021_pais = titulos_2021.groupby("country")["title"].count()
titulos_2021_pais

country
Argentina                        2
Australia                        7
Australia, United Kingdom        1
Australia, United States         1
Belgium                          2
                                ..
United States, Mexico            1
United States, Singapore         1
United States, United Kingdom    4
Uruguay, Germany                 1
Vietnam                          1
Name: title, Length: 74, dtype: int64

#### Ejercicio 14: ¿Cuántos títulos fueron dirigidos por cada director? Ordena los resultados de mayor a menor

In [96]:
#agrupamos por director contando los títulos
titulos_director = df_catalogo.groupby("director")["title"].count()
# ordenamos de mayor a menor
titulos_director.sort_values(ascending = False)

director
Rajiv Chilaka                                                 19
Raúl Campos, Jan Suter                                        18
Suhas Kadav                                                   16
Marcus Raboy                                                  16
Jay Karas                                                     14
                                                              ..
Álvaro Delgado-Aparicio L.                                     1
Álvaro Brechner                                                1
Zuko Nodada                                                    1
Zsolt Pálfi                                                    1
Zoya Akhtar, Karan Johar, Anurag Kashyap, Dibakar Banerjee     1
Name: title, Length: 4528, dtype: int64

#### Ejercicio 15: ¿Cuál es el lenguaje más frecuente en los títulos lanzados en los últimos 5 años agrupado por país?

In [105]:
# filtramos por los titulos lanzados en los últimos 5 años
lanzamiento =  pd.Timestamp.now().year -5
titulos_recientes = df_catalogo[df_catalogo["release_year"] > lanzamiento]
# agrupamos por país y lenguaje contando los títulos
titulos_recientes_idioma_pais = df_catalogo.groupby(["country", "Language"])["title"].count()
titulos_recientes_idioma_pais.idxmax()

('United States', 'English')